## Przykłady wzorca Adapter w kodzie Pythona

Wzorzec Adapter **konwertuje niekompatybilne interfejsy** aby umożliwić współpracę.

**Kluczowa cecha**: Adapter tłumaczy wywołania z jednego interfejsu na drugi - interfejsy są niekompatybilne.

### 1. io.TextIOWrapper - adaptacja binary → text

Najbardziej znany przykład Adapter w Pythonie.

#### Czym jest strumień (stream)?

**Strumień** to sposób na czytanie/zapisywanie danych kawałek po kawałku (zamiast wszystko naraz).

Analogia: oglądasz serię na Netflixie - nie pobierasz całego sezonu naraz, tylko "strumieniujesz" odcinek za odcinkiem.

**Dwa typy strumieni:**

1. **Strumień binarny** - operuje na surowych bajtach (`bytes`):
   - Jak przepisywanie książki bajt po bajcie, nie rozumiejąc treści
   - Używany do: plików, sieci (socket), dysku
   - Typ danych: `bytes` - np. `b"hello"` (bajty reprezentujące tekst)

2. **Strumień tekstowy** - operuje na znakach (`str`):
   - Jak czytanie książki znak po znaku, rozumiejąc treść
   - Używany przez: kod aplikacji (łatwiej pracować)
   - Typ danych: `str` - np. `"hello"` (normalny tekst)

**Problem**: Plik/sieć daje nam `bytes`, ale my chcemy pracować na `str`.

**Rozwiązanie**: `TextIOWrapper` - adapter który tłumaczy `bytes` ↔ `str`.

#### Niekompatybilne interfejsy

Interfejs binarny vs tekstowy - **nie można ich mieszać**:

In [ ]:
import io

# Interfejs binarny - operuje na bytes
binary_stream = io.BytesIO()


In [ ]:
# Zapiszmy bajty do strumienia
print("Interfejs binarny (Adaptee):")
binary_stream.write(b"bytes")  # ✓ działa
print("  binary_stream.write(b'bytes') - OK")

In [ ]:
# Zapiszmy str do strumienia
binary_stream.write("str")  # ✗ TypeError

Binary stream NIE ma interfejsu tekstowego.

#### Adapter: TextIOWrapper

`TextIOWrapper` adaptuje binary stream do interfejsu tekstowego:

In [ ]:
# Adapter: TextIOWrapper adaptuje binary → text
binary_stream = io.BytesIO()
text_adapter = io.TextIOWrapper(binary_stream, encoding='utf-8')

In [ ]:
print("Interfejs tekstowy (przez Adapter):")
text_adapter.write("tekst")  # ✓ działa - adapter konwertuje str → bytes
print("  text_adapter.write('tekst') - OK")

In [ ]:
text_adapter.write(b"bytes")  # ✗ TypeError

print(f"  text_adapter.encoding: {text_adapter.encoding}")
print("  \nText adapter MA interfejs tekstowy!")

#### Jak działa adaptacja?

Adapter tłumaczy wywołania tekstowe na binarne:

In [ ]:
# Demonstracja: co dzieje się pod spodem
binary_stream = io.BytesIO()
text_adapter = io.TextIOWrapper(binary_stream, encoding='utf-8')

# Klient pisze tekst
text_adapter.write("Witaj świecie! 🐍")
text_adapter.write("\nDruga linia")
text_adapter.flush()  # Wymuś zapis

# Sprawdzenie: co jest w binary stream?
binary_stream.seek(0)
binary_data = binary_stream.read()

print("Co klient zapisał (interfejs tekstowy):")
print("  text_adapter.write('Witaj świecie! 🐍')")
print("  text_adapter.write('\\nDruga linia')")

print("\nCo jest w binary stream (pod spodem):")
print(f"  Typ: {type(binary_data)}")
print(f"  Dane: {binary_data}")
print("\nAdapter przekonwertował str → bytes (UTF-8)!")

#### Odczyt przez adapter

In [ ]:
# Odczyt z powrotem jako tekst
binary_stream.seek(0)  # cofamy do początku (jak VHS)
text_adapter.seek(0)
text_data = text_adapter.read()

print("Odczyt przez adapter (interfejs tekstowy):")
print(f"  Typ: {type(text_data)}")
print(f"  Dane: {repr(text_data)}")
print("\nAdapter przekonwertował bytes → str (UTF-8)!")

#### Mapowanie na wzorzec Adapter

**Struktura:**
- **Target**: `TextIOBase` - interfejs tekstowy (`write(str)`, `read() -> str`, `encoding`)
- **Adaptee**: `BufferedIOBase`/`BytesIO` - interfejs binarny (`write(bytes)`, `read() -> bytes`)
- **Adapter**: `TextIOWrapper` - tłumaczy wywołania tekstowe na binarne
- **Client**: kod używający interfejsu tekstowego

**Adaptacja:**
- `write(str)` → encoding → `write(bytes)`
- `read() -> bytes` → decoding → `read() -> str`
- Dodaje `encoding`, `errors`, `newline` - funkcjonalność tekstowa

#### Praktyczne zastosowanie

`TextIOWrapper` jest używany w wielu miejscachw Pythonie:

In [ ]:
import sys

# 1. open() używa TextIOWrapper
with open('test.txt', 'w') as f:
    print(f"open() zwraca: {type(f)}")
    print(f"  encoding: {f.encoding}")
    f.write("tekst")  # interfejs tekstowy

# 2. sys.stdout to TextIOWrapper
print(f"\nsys.stdout type: {type(sys.stdout)}")
print(f"  encoding: {sys.stdout.encoding}")

# 3. Można stworzyć ręcznie dla socket, pipe, etc.
binary_stream = io.BytesIO()
text_file = io.TextIOWrapper(binary_stream, encoding='utf-8')
print(f"\nRęczny TextIOWrapper: {type(text_file)}")

print("\nWszystkie używają tego samego interfejsu tekstowego!")

#### Dlaczego to jest Adapter?

Spełnia wszystkie kryteria wzorca:

1. **Niekompatybilne interfejsy**:
   - Binary: `write(bytes)`, `read() -> bytes`
   - Text: `write(str)`, `read() -> str`
   - NIE MOŻNA ich mieszać (TypeError)

2. **Adapter konwertuje interfejs**:
   - Tłumaczy `str` → `bytes` (encoding)
   - Tłumaczy `bytes` → `str` (decoding)
   - Dodaje funkcjonalność tekstową (`encoding`, `newline`)

3. **Klient używa jednego interfejsu**:
   - Kod pracuje tylko na `str`
   - Nie wie że pod spodem są `bytes`
   - Ten sam kod działa z plikiem, socketem, pipe

4. **Kompozycja**:
   - `TextIOWrapper` zawiera binary stream
   - Deleguje operacje po konwersji
   - Adapter obiektowy (nie klasowy)